In [1]:
from dotenv import load_dotenv
import json 
import pandas as pd 
load_dotenv()



True

In [2]:
df = pd.read_json("test_split_benchmark_v0.0.json")

In [12]:
import litellm 

In [45]:
SYS_PROMPT = """
## Task
- You will receive a question that requires temporal and listwise reasoning to be answered. In addition, you will receive the correct answer for the query.
- Your task is to **generate a new question-answer pair** that increases the temporal complexity of the original query. This means you should reformulate the question to include more intricate temporal elements, requiring advanced temporal reasoning to answer.
- Adhere to the answers generated for the original questions when writing the new question and answer set. 
- **The new question should incorporate additional conditions or focus on a specific time range, making it more complex. As a result, the answer may be shorter due to these added constraints.**
- The new question should be more complex in terms of listwise and temporal reasoning. 
- Note that the question should still be answerable, and the answer should be a subset of the original answer. The structure of the answer should be the same as the original answer, namely a list of entities, with years denoting the time span of the entity.
- Fix any parsing errors in the original answer.
 
Provide your response in JSON format, following this schema:
## Output
```json
{
    "question": <new question>,
    "answers": <list of answers>,
}
```
"""

USR_PROMPT = """
## Question
{question}

## Answeres:
{answers}
"""


In [17]:
from litellm import batch_completion
from litellm import completion
import json_repair

In [49]:
usr_prompt = USR_PROMPT.format(question=df['question'][2], answers=df['answers'][2])

test = completion(model="gpt-4o-mini", messages=[{"role": "system", "content": SYS_PROMPT}, {"role": "user", "content": usr_prompt}], temperature=0.1)


In [30]:
df['questionType'] = df['question'].apply(lambda x: 'sports' if 'teams' in x else 'political')
df.head()


,question,answers,subject,wikidata_ID,aliases,questionType
0,List all teams John Nutter played to this day.,"{'Wycombe Wanderers F.C.': '2001', 'Blackburn ...",John Nutter,Q6250736,[],sports
1,List all political positions Brian Pallister ...,"{'22nd Premier of Manitoba': '2016-2021', 'Lea...",Brian Pallister,Q2924976,[Brian William Pallister],political
2,List all political positions Rob Nicholson he...,{'Shadow Minister of Justice Shadow Attorney G...,Rob Nicholson,Q1566929,[Robert Nicholson],political
3,List all teams Ben May played to this day.,"{'Millwall F.C.': '2001-2008, 2000-2001', 'Sou...",Ben May,Q2895519,"[Benjamin Steven May, Benjamin May]",sports
4,List all teams Steven Caulker played to this ...,"{'Tottenham Hotspur F.C.': '2009-2013', 'Totte...",Steven Caulker,Q26192,[Steven Roy Caulker],sports


In [92]:
SYS_PROMPT_A1 = \
"""
## Task
- You will receive an original question along with its correct answer. The question asks for a list of entities associated with a person, including temporal information.
- Note that the temporal information will be given in the format of years, or years intervals. The years intervals are inclusive of the start and end year.
- Your task is to **generate a new question-answer pair** by rephrasing the original question to include a specific time interval condition, such as "between 2000 and 2010", "before 2000", "after 2010".
- Select a time interval that includes as many entities as possible from the original answer (i.e., maximize overlap with the timelines). The answer should be a subset of the original answer, including only those entities whose time spans overlap with the selected interval.
- The new question should require temporal reasoning to answer due to the added time constraints.
- The answer should maintain the same format as the original answer: a list of entities with years denoting the time span for each entity.
- Ensure that the new question is clear, natural, and appropriately complex in its temporal conditions.
- Fix any parsing errors in the original answer before producing the new answer.
- Be careful with the temporal interval both in the question and the answer, it should be inclusive of the start and end year.

Provide your response in JSON format, following this schema:
## Output
```json
{
    "question": "<new question>",
    "answers": {
        "<entity_1>": "<years>",
        "<entity_2>": "<years>",
        ...
    }
}
```
"""

SYS_PROMPT_A2 = \
"""
## Task
- You will receive an original question along with its correct answer. The question asks for a list of entities associated with a person, including temporal information.
- Your task is to **generate a new question-answer pair** by rephrasing the original question in a more indirect way, incorporating a temporal marker based on one of the entities from the original answer.
- The new question should reference an event or time associated with an entity from the original answer (e.g., "after he left [Entity]", "before joining [Entity]", "following his tenure at [Entity]"). Then ask for entities that are before or after that temporal marker.
- The answer should be a subset of the original answer, including only those entities that satisfy the new temporal conditions specified in the question.
- The new question should require temporal reasoning to answer due to the indirect temporal references.
- The new question should not contain a time span, only the temporal marker.
- The answer should maintain the same format as the original answer: a list of entities with years denoting the time span for each entity.
- Ensure that the new question is clear, natural, and appropriately complex in its temporal conditions.
- Fix any parsing errors in the original answer before producing the new answer.

Provide your response in JSON format, following this schema:
## Output
```json
{
    "question": "<new question>",
    "answers": {
        "<entity_1>": "<years>",
        "<entity_2>": "<years>",
        ...
    }
}
```
"""

USR_PROMPT = """
## Question
{question}

## Answers:
{answers}
"""


In [86]:
print(usr_prompt )


## Question
List all political positions  Rob Nicholson held to this day.

## Answeres:
{'Shadow Minister of Justice Shadow Attorney General of Canada': '2015-2019', 'Minister of Foreign Affairs': 2015, 'Minister of National Defence': '2013-2015', '[[Minister of Justice (Canada)|Minister of Justice\nAttorney General of Canada]]': '2007-2013', 'Leader of the Government in the House of Commons': '2006-2007', 'Minister responsible for Democratic Reform': '2006-2007, 2004-2019, 1984-1993'}



In [105]:
usr_prompt = USR_PROMPT.format(question=df['question'][9], answers=df['answers'][9])
test = completion(model="gpt-4o-2024-11-20", messages=[{"role": "system", "content": SYS_PROMPT_A1}, {"role": "user", "content": usr_prompt}], temperature=0.2)
test2 = completion(model="bedrock/anthropic.claude-3-5-sonnet-20241022-v2:0", messages=[{"role": "system", "content": SYS_PROMPT_A2}, {"role": "user", "content": usr_prompt}], temperature=0.2, aws_region_name="us-west-2")
print(test.choices[0].message.content)
print(test2.choices[0].message.content)



```json
{
    "question": "Which political positions did Pedro Cateriano hold between 2012 and 2016?",
    "answers": {
        "Prime Minister of Peru": "2015-2016",
        "Minister of Defense": "2012-2015"
    }
}
```
{
    "question": "What political positions did Pedro Cateriano hold after his time as Deputy Minister of Justice?",
    "answers": {
        "Minister of Defense": "2012-2015",
        "Prime Minister of Peru": "2015-2016, 2020"
    }
}


In [106]:
batch_messages_1 = []
batch_messages_2 = []

for i in range(len(df)):
    usr_prompt = USR_PROMPT.format(question=df['question'][i], answers=df['answers'][i])
    batch_messages_1.append([{"role": "system", "content": SYS_PROMPT_A1}, {"role": "user", "content": usr_prompt}])
    batch_messages_2.append([{"role": "system", "content": SYS_PROMPT_A2}, {"role": "user", "content": usr_prompt}])

In [114]:
batch_completions_1 = batch_completion(model="gpt-4o-2024-11-20", messages=batch_messages_1, temperature=0.2)

In [115]:
batch_completions_2 = batch_completion(model="gpt-4o-2024-11-20", messages=batch_messages_2, temperature=0.2)



In [116]:
import json_repair

In [117]:
batch_completions_1_parsed = [json_repair.repair_json(completion.choices[0].message.content) for completion in batch_completions_1]
batch_completions_2_parsed = [json_repair.repair_json(completion.choices[0].message.content) for completion in batch_completions_2]


In [122]:
df['time_interval_entries'] = batch_completions_1_parsed
df['temporal_marker_entries'] = batch_completions_2_parsed



,question,answers,subject,wikidata_ID,aliases,questionType,time_interval_entries,temporal_marker_entries
0,List all teams John Nutter played to this day.,"{'Wycombe Wanderers F.C.': '2001', 'Blackburn ...",John Nutter,Q6250736,[],sports,"{""question"": ""Which teams did John Nutter play...","{""question"": ""Which teams did John Nutter play..."
1,List all political positions Brian Pallister ...,"{'22nd Premier of Manitoba': '2016-2021', 'Lea...",Brian Pallister,Q2924976,[Brian William Pallister],political,"{""question"": ""What political positions did Bri...","{""question"": ""What political positions did Bri..."
2,List all political positions Rob Nicholson he...,{'Shadow Minister of Justice Shadow Attorney G...,Rob Nicholson,Q1566929,[Robert Nicholson],political,"{""question"": ""What political positions did Rob...","{""question"": ""Which political positions did Ro..."
3,List all teams Ben May played to this day.,"{'Millwall F.C.': '2001-2008, 2000-2001', 'Sou...",Ben May,Q2895519,"[Benjamin Steven May, Benjamin May]",sports,"{""question"": ""Which teams did Ben May play for...","{""question"": ""Which teams did Ben May play for..."
4,List all teams Steven Caulker played to this ...,"{'Tottenham Hotspur F.C.': '2009-2013', 'Totte...",Steven Caulker,Q26192,[Steven Roy Caulker],sports,"{""question"": ""Which teams did Steven Caulker p...","{""question"": ""Which teams did Steven Caulker p..."
5,List all teams Mitchell Johnson played to thi...,{'Australia national cricket team': '2005-2015...,Mitchell Johnson,Q3180617,[Mitchell Guy Johnson],sports,"{""question"": ""Which teams did Mitchell Johnson...","{""question"": ""Which teams did Mitchell Johnson..."
6,List all political positions Eric Chu held to...,"{'Chairman of the Kuomintang': '2021-2024, 201...",Eric Chu,Q5386280,"[Chu Li-lun, Zhu Lilun, Eric Chu Li-luan, Chu ...",political,"{""question"": ""List all political positions Eri...","{""question"": ""What political positions did Eri..."
7,List all political positions Peter MacKay hel...,{'[[Minister of Justice (Canada)|Minister of J...,Peter MacKay,Q716246,[Peter Gordon MacKay],political,"{""question"": ""What political positions did Pet...","{""question"": ""What political positions did Pet..."
8,List all teams Liam Trotter played to this day.,"{'Ipswich Town F.C.': '2006-2010, -2006', 'Mil...",Liam Trotter,Q542371,[],sports,"{""question"": ""Which teams did Liam Trotter pla...","{""question"": ""Which teams did Liam Trotter pla..."
9,List all political positions Pedro Cateriano ...,"{'Prime Minister of Peru': '2020, 2015-2016', ...",Pedro Cateriano,Q2917191,[Pedro Cateriano Bellido],political,"{""question"": ""Which political positions did Pe...","{""question"": ""Which political positions did Pe..."


In [123]:
df.to_json("new_entries_test_set.jsonl", orient="records", lines=True)

In [124]:
test_new_df = pd.read_json("new_entries_test_set.jsonl", lines=True, orient="records")



,question,answers,subject,wikidata_ID,aliases,questionType,time_interval_entries,temporal_marker_entries
0,List all teams John Nutter played to this day.,"{'Wycombe Wanderers F.C.': '2001', 'Blackburn ...",John Nutter,Q6250736,[],sports,"{""question"": ""Which teams did John Nutter play...","{""question"": ""Which teams did John Nutter play..."
1,List all political positions Brian Pallister ...,"{'22nd Premier of Manitoba': '2016-2021', 'Lea...",Brian Pallister,Q2924976,[Brian William Pallister],political,"{""question"": ""What political positions did Bri...","{""question"": ""What political positions did Bri..."
2,List all political positions Rob Nicholson he...,{'Shadow Minister of Justice Shadow Attorney G...,Rob Nicholson,Q1566929,[Robert Nicholson],political,"{""question"": ""What political positions did Rob...","{""question"": ""Which political positions did Ro..."
3,List all teams Ben May played to this day.,"{'Millwall F.C.': '2001-2008, 2000-2001', 'Sou...",Ben May,Q2895519,"[Benjamin Steven May, Benjamin May]",sports,"{""question"": ""Which teams did Ben May play for...","{""question"": ""Which teams did Ben May play for..."
4,List all teams Steven Caulker played to this ...,"{'Tottenham Hotspur F.C.': '2009-2013', 'Totte...",Steven Caulker,Q26192,[Steven Roy Caulker],sports,"{""question"": ""Which teams did Steven Caulker p...","{""question"": ""Which teams did Steven Caulker p..."
5,List all teams Mitchell Johnson played to thi...,{'Australia national cricket team': '2005-2015...,Mitchell Johnson,Q3180617,[Mitchell Guy Johnson],sports,"{""question"": ""Which teams did Mitchell Johnson...","{""question"": ""Which teams did Mitchell Johnson..."
6,List all political positions Eric Chu held to...,"{'Chairman of the Kuomintang': '2021-2024, 201...",Eric Chu,Q5386280,"[Chu Li-lun, Zhu Lilun, Eric Chu Li-luan, Chu ...",political,"{""question"": ""List all political positions Eri...","{""question"": ""What political positions did Eri..."
7,List all political positions Peter MacKay hel...,{'[[Minister of Justice (Canada)|Minister of J...,Peter MacKay,Q716246,[Peter Gordon MacKay],political,"{""question"": ""What political positions did Pet...","{""question"": ""What political positions did Pet..."
8,List all teams Liam Trotter played to this day.,"{'Ipswich Town F.C.': '2006-2010, -2006', 'Mil...",Liam Trotter,Q542371,[],sports,"{""question"": ""Which teams did Liam Trotter pla...","{""question"": ""Which teams did Liam Trotter pla..."
9,List all political positions Pedro Cateriano ...,"{'Prime Minister of Peru': '2020, 2015-2016', ...",Pedro Cateriano,Q2917191,[Pedro Cateriano Bellido],political,"{""question"": ""Which political positions did Pe...","{""question"": ""Which political positions did Pe..."
